<a href="https://colab.research.google.com/github/Jwilson1172/lambda/blob/master/unit-1-build/clean_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#used to change all the relive dates to datetimes
import datetime

# this in important because the time stamps in the dataset are relitive to when they were scraped
DATE_SCRAPED="02-24-2020"

In [0]:
# read in the data frame that has my scrapped content
df=pd.read_csv('/content/google_reviews_chanellos.csv')
df_labels=pd.read_csv('/content/chanellos_locations.csv')


# **EDA**

In [96]:
# inspect head @TODO DROP NAME COL
print(df.shape)
df.head()

(2096, 7)


,web-scraper-order,web-scraper-start-url,number_to_rate,name,published_time,review_content,rating_html
0,1582527995-344,https://www.google.com/maps/place/Chanello's+P...,124 reviews,lauren sams,2 weeks ago,Great and fast service with excellent food.,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
1,1582527995-428,https://www.google.com/maps/place/Chanello's+P...,124 reviews,Krys Peirce,a year ago,NaN,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
2,1582527918-244,https://www.google.com/maps/place/Chanello's+P...,101 reviews,lori kent,3 years ago,NaN,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
3,1582528346-1348,https://www.google.com/maps/place/Chanello's+P...,62 reviews,Christian Betz,4 years ago,NaN,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
4,1582527995-377,https://www.google.com/maps/place/Chanello's+P...,124 reviews,Janel Brosseit,a year ago,This rating goes more towards the food. Just r...,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."


In [117]:
df_labels.head()
df_labels=df_labels.drop('franchise') # there are no franchised stores in this list


,web-scraper-order,name,rating,address,num_to_rate,start_url,franchise
0,1582755020-11,Chanello's Pizza #1,3.7,8245 Hampton Blvd,87,https://www.google.com/maps/place/Chanello's+P...,False
1,1582755020-2,Chanello's Pizza #19,4.0,2226 Pleasure House Rd,68,https://www.google.com/maps/place/Chanello's+P...,False
2,1582755020-16,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,https://www.google.com/maps/place/Chanello's+P...,False
3,1582755020-1,Chanello's Pizza #2,3.1,3201 E Ocean View Ave,116,https://www.google.com/maps/place/Chanello's+P...,False
4,1582755020-6,Chanello's Pizza #11,3.4,6657 Indian River Rd,148,https://www.google.com/maps/place/Chanello's+P...,False


In [131]:
def strip_n2r(x):
  if isinstance(x,float):
    return x
  else:
    return x[:-8]
test=df['number_to_rate'].apply(strip_n2r)
print(test.head(40))
print(test.dtypes,test.shape)
test.isnull().sum()
#df['number_to_rate'].isin(df_labels['num_to_rate']).sum()

0     124
1     124
2     101
3      62
4     124
5     111
6     124
7     101
8      83
9      83
10    141
11     87
12    101
13    207
14    156
15     87
16    NaN
17    207
18    207
19    143
20    111
21    101
22    120
23    111
24    143
25     83
26    156
27     64
28    141
29    124
30    NaN
31    156
32     85
33     68
34    156
35    207
36     90
37     64
38     83
39    111
Name: number_to_rate, dtype: object
object (2096,)


264

In [134]:
((-(2096-264)/2096)+1)*100

12.595419847328248

#Data Cleaning/feature engineering

###**geo-codes engineering**

In [0]:
# i have two options here i can just assign new values to each store and call it a day
# or I noticed that theese links have geotags on them, i can pull the geotag from the links
# then trash them
df['web-scraper-start-url'].value_counts()

https://www.google.com/maps/place/Chanello's+Pizza+%239/@36.892055,-76.2405181,17z/data=!3m1!4b1!4m10!1m2!2m1!1sChanello's+Pizza!3m6!1s0x89ba9710c59fbbc9:0x4b7bd022c70b0edb!8m2!3d36.8920507!4d-76.2383294!9m1!1b1                                           207
https://www.google.com/maps/place/Chanello's+Pizza+%235/@36.8715743,-76.3002947,17z/data=!3m1!4b1!4m10!1m2!2m1!1sChanello's+Pizza!3m6!1s0x89ba984644f0319f:0xe26b261878364c00!8m2!3d36.87157!4d-76.298106!9m1!1b1                                             156
https://www.google.com/maps/place/Chanello's+Pizza+%2311/@36.8190953,-76.2241417,17z/data=!3m1!4b1!4m10!1m2!2m1!1sChanello's+Pizza!3m6!1s0x89babd9dace7bc81:0x26dac2e722fbf6b3!8m2!3d36.819091!4d-76.221953!9m1!1b1                                           148
https://www.google.com/maps/place/Chanello's+Pizza+%233/@36.8409833,-75.9968367,17z/data=!3m2!4b1!5s0x89baea29ba7bd3c3:0x6a010471d2c98ca0!4m10!1m2!2m1!1sChanello's+Pizza!3m6!1s0x89baea2998386b9d:0x549dc79761d1f91b!8m2!3d36.840

In [0]:
# define a function to strip the url strings to reveal geo tags
def parse_geo(x):
  return x.split('@')[1].split(',')[:-1]
# test it out before i make any changes
test=df['web-scraper-start-url'].apply(parse_geo)
test.head()

0    [36.7819861, -76.3383989]
1    [36.7819861, -76.3383989]
2    [36.8262133, -76.4123802]
3    [36.8031384, -76.1770067]
4    [36.7819861, -76.3383989]
Name: web-scraper-start-url, dtype: object

In [0]:
# make changes
df['geo']=df['web-scraper-start-url'].apply(parse_geo)

###**rating valuation engineering**

In [0]:
# on the reviews it had images of lit stars triggered by js
# to work around this i took the html source because i knew it would
# show which stars are supossed to be triggered, but as a drawback
# no i have to clean HTML instead of pretty output
df['rating_html'].head()

0    <span jstcache="293" jsinstance="0" class="sec...
1    <span jstcache="293" jsinstance="0" class="sec...
2    <span jstcache="293" jsinstance="0" class="sec...
3    <span jstcache="293" jsinstance="0" class="sec...
4    <span jstcache="293" jsinstance="0" class="sec...
Name: rating_html, dtype: object

In [0]:
# make a funtion to apply to col
def clean_rating(x):
  # make a list of strings to work with
  out=x.split()
  # sort list so that duplicates group togeather
  out.sort()
  # the first element is just '<span>' and the following 20 elements are redundant,
  # the last 5 elements are placment flags
  # remove all of them and leave me with just the number of active stars
  # then get a count so i can have a pretty int
  return len(out[21:len(out)-5])

# test to make sure that it works
test=df['rating_html'].apply(clean_rating)
test.head()

0    5
1    1
2    4
3    4
4    2
Name: rating_html, dtype: int64

In [0]:
# make changes to df
df['rating_html']=df['rating_html'].apply(clean_rating)

### **datetime engeenering for graphing**

In [0]:
# humanfied dates are great to read, not so great to graph, i need to make theese
# back into dt format,(why i have a glob at the top with the date of scrape)
df['published_time'].value_counts()

a year ago       628
2 years ago      394
3 years ago      168
7 months ago      91
8 months ago      90
6 months ago      74
4 years ago       72
11 months ago     68
5 months ago      67
9 months ago      65
10 months ago     56
a month ago       55
4 months ago      54
3 months ago      47
2 months ago      35
5 years ago       26
a week ago        18
2 weeks ago       17
6 years ago       16
3 weeks ago       12
4 weeks ago       11
8 years ago       10
7 years ago        9
5 days ago         4
a day ago          4
6 days ago         2
9 years ago        2
3 days ago         1
Name: published_time, dtype: int64

In [0]:
# create a map for all units to hours
time_map={'second':0.000278,'minute':0.0167,'hour':1,'day':24,'week':168,'month':530.5,'year':8766}
# make a function that takes a relitive time 'a day ago' and transfers that to a datetime
def human_to_dt(x):
  # the number that i will end up subtracting from scrape date
  diff=0
  # make a datetime object to hold the date that the dataset was scrapped
  dt=datetime.datetime.strptime(DATE_SCRAPED, '%m-%d-%Y')
  # strip the string down to two values a quantifier and a unit
  q,u=test[:-3].strip().split(' ')
  # remove trailing s on unit it is not needed
  if u[len(u)-1]=="s":
    u=u[:len(u)-1]
  # check if there is just one unit if so then set diff=1
  if q == "a":
    diff=time_map[u]
  # if the number is not one then multiply the q by the map_key entry for u
  else:
    #safe cast
    try:
      diff=time_map[u]*int(q)
    except:
      print("ERROR")
      return 0
  # convert dt to utc timestamp
  dt=dt-datetime.timedelta(hours=diff)
  timestamp = dt.replace(tzinfo=datetime.timezone.utc).timestamp()
  return timestamp

In [115]:
# apply changes to df
df['published_time']=df['published_time'].apply(human_to_dt)

0    1.550945e+09
1    1.550945e+09
2    1.550945e+09
3    1.550945e+09
4    1.550945e+09
Name: published_time, dtype: float64